In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension ('tabulator')
import hvplot.pandas

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv')

In [3]:
df

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,AFG,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.165,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
2,Afghanistan,1852,AFG,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
3,Afghanistan,1853,AFG,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
4,Afghanistan,1854,AFG,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.163,0.000,0.000,0.000,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50593,Zimbabwe,2017,ZWE,14751101.0,2.194784e+10,0.469,0.032,9.596,-0.937,-8.899,...,NaN,0.114,0.001,0.001,0.002,0.0,115.59,28.30,0.910,9.486
50594,Zimbabwe,2018,ZWE,15052191.0,2.271535e+10,0.558,0.037,11.795,2.199,22.920,...,NaN,0.114,0.001,0.001,0.002,0.0,118.22,30.83,0.771,6.537
50595,Zimbabwe,2019,ZWE,15354606.0,NaN,0.570,0.037,11.115,-0.681,-5.772,...,NaN,0.113,0.001,0.001,0.002,0.0,117.96,30.53,0.978,8.795
50596,Zimbabwe,2020,ZWE,15669663.0,NaN,0.570,0.036,10.608,-0.507,-4.559,...,NaN,0.112,0.001,0.001,0.002,0.0,NaN,NaN,1.006,9.481


In [4]:
df.columns

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

Data preprocessing

In [5]:
# Fill NAs with Os and create GDP per capita column
df = df. fillna(0)
df ['gdp_per_capita'] = np.where(df ['population']!= 0, df ['gdp']/ df ['population'], 0)

In [6]:
#make dataframe interactive
idf=df.interactive()

CO2 EMISSIONS BY CONTINENT

In [7]:
# Define Panel widgets
year_slider = pn.widgets. IntSlider (name= 'Year slider', start= 1750, end=2021, step=5, value=1850)
year_slider

IntSlider(end=2021, name='Year slider', start=1750, step=5, value=1850)

In [8]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets. RadioButtonGroup (
name='Y axis',
options= ['co2', 'co2_per_capita',],
button_type='success')

In [9]:
continents = ['World',
'Asia','Oceania',
'Europe', 'Africa', 'North America',
'South America',
'Antarctica']
co2_pipeline = (
idf [
    (idf.year <= year_slider) &
(idf.country.isin(continents))
]
.groupby (['country', 'year']) [yaxis_co2]. mean ()
.to_frame ()
.reset_index()
.sort_values (by='year')
.reset_index(drop=True)
)


In [10]:
co2_pipeline

In [11]:
co2_plot = co2_pipeline. hvplot(x = 'year', by='country', y=yaxis_co2, line_width=2, title="CO2 emission by continent")
co2_plot

(2) Table - CO2 emission over time by continent

In [12]:

co2_table = co2_pipeline.pipe (pn.widgets. Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
co2_table

(3) CO2 vs GDP scatterplot

In [13]:
co2_vs_gdp_scatterplot_pipeline = (
idf[
    (idf.year == year_slider) &
(~ (idf.country.isin (continents)))
]
.groupby (['country', 'year', 'gdp_per_capita']) ['co2' ].mean ()
.to_frame ()
. reset_index ()
.sort_values (by='year')
. reset_index (drop=True)
)


In [14]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita',
y='co2',
by='country',
size=80, kind="scatter",
alpha=0.7,
legend= False,
height=500,
width=500)
co2_vs_gdp_scatterplot

(4) Bar chart with CO2 sources by continent

In [15]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
name='Y axis',
options=['coal_co2', 'oil_co2', 'gas_co2'],
button_type='success'
)
continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']
co2_source_bar_pipeline = (idf[
(idf.year == year_slider)&
(idf.country.isin(continents_excl_world))
 ]
.groupby ( ['year', 'country']) [yaxis_co2_source] .sum()
.to_frame()
.reset_index()
.sort_values(by='year')
.reset_index(drop=True)
)

In [16]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', x='country', y=yaxis_co2_source, title='CO2 source by continent')
co2_source_bar_plot

CREATING DASHBOARD

In [18]:
#Layout using Template
template = pn. template. FastListTemplate(
     title='World CO2 emission dashboard',
     sidebar= [pn.pane.Markdown ("# C02 Emissions and Climate Change"),
               pn.pane.Markdown ("####Carbon dioxide emissions are the primary driver of global climate change. It's widelyrecognised that to avoid the worst impacts ofclimate change, the world needs to urgentlyreduce emissions. But, how this responsibilityis shared between regions, countries, andindividuals has been an endless point ofcontention in internationa discussions"),
               pn. pane. PNG( 'climate_day.png', sizing_mode='scale_both'),
               pn. pane. Markdown ("## Settings"),
               year_slider],
      main= [pn. Row (pn. Column (yaxis_co2,
                            co2_plot.panel(width=700), margin= (0,25)),
                            co2_table.panel(width=500)),
                pn. Row(pn. Column (co2_vs_gdp_scatterplot.panel(width=600), margin= (0,25)),
                pn. Column(yaxis_co2_source, co2_source_bar_plot.panel(width=600)))],
      accent_base_color="#88d8b0",
      header_background="#88d8b0",
    ) 
# template.show()
template. servable ();